In [1]:
## import packages
from util import createGrps
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms import Predictor
from lenskit.algorithms.user_knn import UserUser
from lenskit import batch
import sys

In [2]:
## Get the training data and train the user-user collaborative filterring
# read the data (about applications)
# TODO: replace by Tine's filter  data
data = pd.read_csv('../dataset/user_ratings_neg_1000_20_20_1.csv', delimiter=',')

# load user data
user_data = pd.read_csv('../dataset/users.tsv', sep='\t')

In [3]:
def cap(x):
    if x > 1.0:
        return 1.0
    elif x < 0.0:
        return 0.0
    return x 

In [4]:
## First get the groups
# parameters for group generation
size = 10
grpNum = 20
seed = 12345

# generate synthetic groups of users and display them
groups2 = createGrps(pd.read_csv('../dataset/users.tsv', sep='\t'), 10, 10, 12345)
display(groups2)

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,Not Applicable,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
2,178,1,Train,Greenville,SC,US,29609,High School,Not Applicable,Not Applicable,6,35.0,No,Yes,4
3,344,1,Train,Newport News,VA,US,23601,High School,Not Applicable,2007-01-01 00:00:00,3,7.0,Yes,No,0
4,496,1,Train,Easley,SC,US,29640,High School,Not Applicable,1987-01-01 00:00:00,10,21.0,No,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93300,1471765,7,Train,Kennesaw,GA,US,30144,High School,Not Applicable,2012-06-01 00:00:00,1,2.0,No,No,0
93301,1471878,7,Train,Orangeburg,SC,US,29115,High School,Not Applicable,1999-01-01 00:00:00,4,3.0,No,No,0
93302,1471997,7,Train,Baltimore,MD,US,21222,High School,Math,2004-06-01 00:00:00,3,5.0,No,Yes,45
93303,1472042,7,Train,Saint Clair Shores,MI,US,48081,High School,Not Applicable,1970-01-01 00:00:00,7,32.0,Yes,No,0


,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
1,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
2,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0
3,131,1,Train,Houston,TX,US,77077,Bachelor's,Finance,1998-05-01 00:00:00,3,14.0,Not Applicable,No,0
4,162,1,Train,Long Beach,CA,US,90807,Master's,I/O Psychology,2012-05-01 00:00:00,10,25.0,No,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196245,1471622,7,Train,Jersey City,NJ,US,07307,Bachelor's,Psychology,2012-06-01 00:00:00,4,8.0,Yes,No,0
196246,1471633,7,Train,Sacramento,CA,US,95823,Bachelor's,Deaf Education,1985-01-01 00:00:00,9,14.0,Yes,Yes,30
196247,1471828,7,Train,Atlanta,GA,US,30312,PhD,Not Applicable,2006-05-01 00:00:00,3,6.0,Yes,No,0
196248,1472058,7,Train,Maspeth,NY,US,11378,Associate's,Not Applicable,2001-01-01 00:00:00,2,8.0,Yes,Yes,3


,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,411,1,Train,Lutz,FL,US,33559,Not Applicable,Not Applicable,Not Applicable,3,11.0,Yes,Yes,1
1,640,1,Train,Cooper City,FL,US,33330,Not Applicable,Not Applicable,Not Applicable,4,34.0,Not Applicable,No,0
2,1113,1,Train,Ramsey,NJ,US,07446,Not Applicable,Not Applicable,2011-10-01 00:00:00,2,1.0,No,No,0
3,1123,1,Train,Jacksonville,FL,US,32256,Not Applicable,Civil Engineering and Architecture,2012-12-01 00:00:00,3,9.0,Yes,Yes,100
4,1226,1,Train,Cincinnati,OH,US,45245,Not Applicable,Licensed Practical/Vocational Nurse Training (...,1988-12-01 00:00:00,8,23.0,No,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100148,1471719,7,Train,Baltimore,MD,US,21216,Not Applicable,Customer service,2008-01-01 00:00:00,0,Not Applicable,No,No,0
100149,1471870,7,Train,Lubbock,TX,US,79412,Not Applicable,Criminal Justice,2003-01-01 00:00:00,5,19.0,No,No,0
100150,1471877,7,Train,Greer,SC,US,29650,Not Applicable,Business Administration,1988-01-01 00:00:00,9,30.0,Not Applicable,No,0
100151,1471901,7,Train,Port Clinton,OH,US,43452,Not Applicable,Not Applicable,Not Applicable,7,25.0,No,No,0


,Users
0,"[632238.0, 153968.0, 701807.0, 244971.0, 12801..."
1,"[522367.0, 1144942.0, 1079883.0, 755100.0, 466..."
2,"[426798.0, 698465.0, 472577.0, 64615.0, 205589..."
3,"[1185537.0, 1101175.0, 757130.0, 1433223.0, 91..."
4,"[226364.0, 338083.0, 92869.0, 75063.0, 1036655..."
5,"[588226.0, 710182.0, 636382.0, 1193664.0, 8769..."
6,"[435846.0, 1387316.0, 782586.0, 959441.0, 1724..."
7,"[848559.0, 597850.0, 845132.0, 2386.0, 413033...."
8,"[889709.0, 1462663.0, 231167.0, 688739.0, 1722..."
9,"[1358184.0, 500736.0, 1455748.0, 159070.0, 116..."


In [ ]:
# construct dataframe in format (user, item, rating) via column addition
df_ui = data.rename(columns={"UserID": "user", "JobID": "item", "Rating":"rating"})
# check data being read properly
display(df_ui.head(10)) 

# train UserUser collaborative filterring
user_user = UserUser(10, min_nbrs=3)  # Minimum (3) and maximum (10) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(df_ui)

In [ ]:
# pd.set_option('display.max_rows', None)
all_ratings = []

## Create a User-Item matrix of scores so we can apply one of the aggregation strategies
# iterate through groups generated
for i, row in groups2.iterrows():
    # get the array of users from the row
    synGrp = row.iloc[0]

    # get recommendations for all group members
    ratings_grp = batch.recommend(recsys, synGrp, n=None,  n_jobs=1)
    
    # cap the values (x > 1 then 1; x < 0 then 0)
    # TODO: if you are confident that you can explain the clamping -> go ahead and use it
    # ratings_grp["score"] = ratings_grp["score"].map(cap)
    # print(ratings_grp)
    # TODO: sys.exit only for demonstration to not run through all 20 groups of users
    all_ratings.append((synGrp, ratings_grp))


In [ ]:
num_pref = 5
group_preference = []
index = 0

for label, ratings_grp in all_ratings:
    pref = []
    # Print the label to differentiate the DataFrames
    print("Label:", label)
    
    # Access and work with the ratings_grp DataFrame
    display(ratings_grp.head(10))  # Example: Display the first few rows of the DataFrame
    
    # Perform any other operations or analysis on the DataFrame as needed
    unique_items = ratings_grp['item'].unique().tolist()

    for item in unique_items:
        job_scores = ratings_grp[ratings_grp['item'] == item]

        
        if len(job_scores) > 1:
            print("Number of job_scores:", len(job_scores))
        
    index += 1
    sys.exit()

NameError: name 'all_ratings' is not defined